In [ ]:
!pip install ultralytics===8.3.213
from google.colab import files
import yaml
from ultralytics import YOLO

## **Download & unzip the requried Datasets**

In [ ]:
!wget "https://www.kaggle.com/api/v1/datasets/download/izeeshan/unified-animal-species-dataset"

In [ ]:
!mkdir -p /content/unified_animal_species_dataset && unzip /content/unified-animal-species-dataset -d /content/unified_animal_species_dataset && rm /content/unified-animal-species-dataset

In [6]:
# Set the path to /content/unified_animal_species_dataset in the dataset.yaml

yaml_path = '/content/unified_animal_species_dataset/dataset.yaml'

with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

data['path'] = '/content/unified_animal_species_dataset'

with open(yaml_path, 'w') as f:
    yaml.safe_dump(data, f, sort_keys=False)


# **Train the Model**

In [ ]:
# Load YOLOv11 model
model = YOLO('yolo11m.pt')

# Train the model
results = model.train(
    data=yaml_path,
    epochs=2,
    imgsz=640,
    batch=16
)

"""
'results' might include:
  results.metrics: final training metrics (mAP, precision, recall, etc.)
  results.save_dir: directory path where all training outputs are stored
  results.model: reference to the trained model
  results.epoch: number of completed epochs
"""

# **Evaluate the model**

In [ ]:
model = YOLO("/content/yolo_dataset/runs/detect/train/weights/best.pt")

# Validate on your dataset (this gives precision, recall, mAP, confusion matrix, etc.)
metrics = model.val(
    data="/content/dataset/meta.yaml",
    split="test",
    imgsz=512
)

print(metrics)  # shows metrics dict

# **Inference**

In [ ]:
# Load your trained model
model = YOLO("/content/yolo_dataset/runs/detect/train/weights/best.pt")

# Upload an image
uploaded = files.upload()

# Run inference (object detection)
for filename in uploaded.keys():
    results = model.predict(
        source=filename,
        save=True, # save annotated image(s)
        conf=0.25, # confidence threshold
        imgsz=640
    )

    print(f"Processed {filename}")


### **Optional Step - Exporting the Trained Model (for Deployment & Portability)**

In [ ]:
# PyTorch format (.pt)
# Export converts it to other formats:
# ONNX → open format, runs on many platforms (good for production).
# TorchScript → optimized PyTorch model (faster inference).
# TensorRT / CoreML → for NVIDIA GPUs / Apple devices.

In [ ]:
# IoU (Intersection over union) = how much the modal drawn box overlapped with the real box (ground truth).
# mAP = your average score across all objects and overlap thresholds.
# Export = translating your model into another “language” so other systems can use it.